<a href="https://colab.research.google.com/github/JuanDavidBoteroRuiz/EntregaMinDat/blob/main/SR_Basado_en_usuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import math
from math import sqrt
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
url_movie = 'https://drive.google.com/uc?id=1H64TOo4OQYnDPn50x1zMkF0bZhqhYaeB'
url_rating = 'https://drive.google.com/uc?id=12vtDPA2TogM4fho6Av8hkcOQI0-BRYDd'

In [ ]:
peliculas = pd.read_csv(url_movie)

In [ ]:
# Se preparan los datos
peliculas['year'] = peliculas.title.str.extract('(\(\d\d\d\d\))',expand = False)
peliculas['year'] = peliculas.year.str.replace('(','').str.replace(')','') 
peliculas['title'] = peliculas.title.str.replace('(\(\d\d\d\d\))','')
peliculas['title'] =peliculas['title'].apply(lambda x: x.strip())
peliculas['genres'] = peliculas.genres.str.split('|') #Ahora se dividen los valores de la columna de generos en una lista de generos para simplificar el uso futuro, lo haremos con una funcion de cadena dividia de python
peliculas = peliculas.drop('genres',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [ ]:
rating = pd.read_csv(url_rating).drop('timestamp',1)
ratings = pd.merge(peliculas,rating).drop('year',1) #.drop('genres',1)
ratings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,movieId,title,userId,rating
0,1,Toy Story,3,4.0
1,1,Toy Story,6,5.0
2,1,Toy Story,8,4.0
3,1,Toy Story,10,4.0
4,1,Toy Story,11,4.5
...,...,...,...,...
1048570,130219,The Dark Knight,1339,4.5
1048571,130462,The Boy,5731,4.0
1048572,130490,Insurgent,2423,1.0
1048573,130490,Insurgent,3397,3.5


In [ ]:
userInput = [
# {'title':'Heat', 'rating':5},
# {'title':'Star Wars: Episode II - Attack of the Clones', 'rating':4.5},
# {'title':'Secret in Their Eyes, The (El secreto de sus ojos)', 'rating':4.5},
# {'title':'Terminator 2: Judgment Day', 'rating':4.5},
# {'title':'Four Rooms', 'rating':3.5}
{'title':'Toy Story', 'rating':3.5}
]
inputMovies = pd.DataFrame(userInput)

In [ ]:
inputId = peliculas[peliculas['title'].isin(inputMovies['title'].tolist())] #Filtrar las películas por título 
inputMovies = pd.merge(inputId, inputMovies) #Luego fusionándolo para que podamos obtener el movieId. Lo está fusionando implícitamente por título. 
inputMovies = inputMovies.drop('year', 1) #Eliminar información que no usaremos del dataframe
inputMovies #Final input dataframe
userSubset = ratings[ratings['movieId'].isin(inputMovies['movieId'].tolist())] #Filtrar a los usuarios que han visto películas que la entrada ha visto y almacenarlas 
userSubset.head() 
userSubsetGroup = userSubset.groupby(['userId']) # Groupby crea varios sub dataframes de datos donde todos tienen el mismo valor en la columna especificada #como parámetro 
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)# Ordenamos por los usuarios que han puntuado mas a las peliculas en común con la entrada
userSubsetGroup = userSubsetGroup[0:100]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
pearsonCorrelationDict = {}
#Para cada grupo de usuarios de nuestro subconjunto 
for name, group in userSubsetGroup:
    #Comencemos ordenando la entrada y el grupo de usuarios actual para que los valores no se mezclen más adelante. 
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obtenga las puntuaciones de las reseñas de las películas que ambos tienen en común 
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #Y luego guárdelos en una variable de búfer temporal en un formato de lista para facilitar cálculos futuros 
    tempRatingList = temp_df['rating'].tolist()
    #Pongamos también las reseñas del grupo de usuarios actual en un formato de lista 
    tempGroupList = group['rating'].tolist()
    data_corr = {'tempGroupList': tempGroupList,
            'tempRatingList': tempRatingList}
    pd_corr = pd.DataFrame(data_corr)
    r = pd_corr.corr(method="pearson")["tempRatingList"]["tempGroupList"]
    #ahora eliminamos los nan de nuestro coef de pearson
    if math.isnan(r) == True:
        r = 0
    pearsonCorrelationDict[name] = r
#Convertimos el diccionario a un dataframe:         
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0,3
1,0,6
2,0,8
3,0,10
4,0,11


In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
0,0,3
63,0,175
73,0,215
72,0,209
71,0,208


In [ ]:
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,title,rating
0,0,3,1,Toy Story,4.0
1,0,3,24,Powder,3.0
2,0,3,32,Twelve Monkeys (a.k.a. 12 Monkeys),4.0
3,0,3,50,"Usual Suspects, The",5.0
4,0,3,160,Congo,3.0


In [ ]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,title,rating,weightedRating
0,0,3,1,Toy Story,4.0,0.0
1,0,3,24,Powder,3.0,0.0
2,0,3,32,Twelve Monkeys (a.k.a. 12 Monkeys),4.0,0.0
3,0,3,50,"Usual Suspects, The",5.0,0.0
4,0,3,160,Congo,3.0,0.0


In [ ]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
5,0,0.0


In [ ]:
#Crea un nuevo dataframe
recommendation_df = pd.DataFrame()
#Ahora tomamos el promedio ponderado 
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,NaN,1
2,NaN,2
3,NaN,3
4,NaN,4
5,NaN,5


In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(30)
print(peliculas.loc[peliculas['movieId'].isin(recommendation_df.head(20)['movieId'].tolist())])

    movieId                           title  year
0         1                       Toy Story  1995
1         2                         Jumanji  1995
2         3                Grumpier Old Men  1995
3         4               Waiting to Exhale  1995
4         5     Father of the Bride Part II  1995
5         6                            Heat  1995
6         7                         Sabrina  1995
8         9                    Sudden Death  1995
9        10                       GoldenEye  1995
10       11         American President, The  1995
11       12     Dracula: Dead and Loving It  1995
12       13                           Balto  1995
13       14                           Nixon  1995
14       15                Cutthroat Island  1995
15       16                          Casino  1995
16       17           Sense and Sensibility  1995
17       18                      Four Rooms  1995
18       19  Ace Ventura: When Nature Calls  1995
19       20                     Money Train  1995
